In [ ]:
import os
import torch
import numpy as np
import random

def seed_everything(seed):
    """
    Set random seed for reproducibility
    """
    # 1. Python & Numpy
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
    # 2. PyTorch (CPU & GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"🔒 Locked Random Seed: {seed}")


In [ ]:



def seed_everything_random():
    """
    Tạo random seed, set seed đó, và return seed để bạn biết
    """
    # Tạo random seed
    random_seed = random.randint(0, 999999)
    
    # Set seed
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    
    # Để reproducible
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    return random_seed


In [ ]:
#import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from preprocess import get_sampler

In [ ]:
#load data
df_men =pd.read_csv(r"C:\Users\Lenovo\Documents\Neu 2025-2026\Lab\Hillstrom-Men.csv")
df_men = df_men.drop(columns="Unnamed: 0")
print ("---------------------------")
print ("null count:")
print (df_men.isnull().sum())
print ("---------------------------")
print(df_men.dtypes)
print ("---------------------------")
print ("labels:")
print(df_men.columns.tolist())
print ("---------------------------")
print("data shape:")
print(df_men.shape)


In [ ]:
#Hillstrom-men
#split num and cate
cate_cols = ['zip_code', 'channel']
df_men["history_segment"] =df_men["history_segment"].map({
    "1) $0 - $100": '1', 
    "2) $100 - $200": "2", 
    "3) $200 - $350": "3",
    "4) $350 - $500": "4",
    "5) $500 - $750": "5",
    "6) $750 - $1,000": "6",
    "7) $1,000 +": "7"                         
})
num_cols = ['recency', 'history_segment']
#split x y t
y_men = df_men["spend"]
t_men = df_men["treatment"]
x_men = df_men.drop(columns=["spend", "treatment", "visit", "conversion", 'history'])

x_men_encode = pd.get_dummies(x_men, columns=cate_cols, drop_first=True)
x_men_encode = x_men_encode.astype(float)
#train test split
x_men_train, x_men_test_val, t_men_train, t_men_test_val, y_men_train, y_men_test_val = train_test_split(x_men_encode,t_men.values, y_men.values, test_size=0.4, random_state=42, stratify=t_men)
x_men_val, x_men_test, t_men_val, t_men_test, y_men_val, y_men_test = train_test_split(x_men_test_val, t_men_test_val, y_men_test_val, test_size= 0.75, random_state=42, stratify=t_men_test_val)

#scale
# scaler = StandardScaler()
# x_men_train[num_cols]= scaler.fit_transform(x_men_train[num_cols])
# # x_men_val[num_cols] = scaler.transform(x_men_val[num_cols])
# x_men_test[num_cols] = scaler.transform(x_men_test[num_cols])

x_men_train = x_men_train.values.astype(float)
x_men_val = x_men_val.values.astype(float)
x_men_test = x_men_test.values.astype(float)
print (x_men_train[:10])


In [ ]:
#Transform to tensor
def to_tensor(df):
    return torch.tensor(df, dtype=torch.float32)

x_men_train_t = to_tensor(x_men_train)
x_men_val_t = to_tensor(x_men_val)
x_men_test_t = to_tensor(x_men_test)

y_men_train_t = to_tensor(y_men_train).unsqueeze(1)
y_men_val_t = to_tensor(y_men_val).unsqueeze(1)
y_men_test_t = to_tensor(y_men_test).unsqueeze(1)

t_men_train_t = to_tensor(t_men_train.astype(float)).unsqueeze(1)
t_men_val_t = to_tensor(t_men_val.astype(float)).unsqueeze(1)
t_men_test_t = to_tensor(t_men_test.astype(float)).unsqueeze(1)


#dual stream 
idx_t = (t_men_train==1)
idx_c = (t_men_train==0)

x_treat = x_men_train_t[idx_t]
t_treat = t_men_train_t[idx_t]
y_treat = y_men_train_t[idx_t]

x_ctrl = x_men_train_t[idx_c]
t_ctrl = t_men_train_t[idx_c]
y_ctrl = y_men_train_t[idx_c]

sampler_treat = get_sampler(y_treat, target_positive_ratio=0.2)
sampler_control = get_sampler(y_ctrl, target_positive_ratio=0.2)
#Data loader
train_t_dataset = TensorDataset(x_men_train_t[idx_t], t_men_train_t[idx_t], y_men_train_t[idx_t])
train_c_dataset = TensorDataset(x_men_train_t[idx_c], t_men_train_t[idx_c], y_men_train_t[idx_c])
val_dataset = TensorDataset(x_men_val_t, t_men_val_t, y_men_val_t)
test_dataset = TensorDataset(x_men_test_t, t_men_test_t, y_men_test_t)

batch_size = 6400
# train_t_loader = DataLoader(train_t_dataset, batch_size= batch_size//2, sampler = sampler_treat, shuffle=False)
# train_c_loader = DataLoader(train_c_dataset, batch_size= batch_size//2, sampler= sampler_control, shuffle= False)
train_t_loader = DataLoader(train_t_dataset, batch_size= batch_size//2, shuffle=True)
train_c_loader = DataLoader(train_c_dataset, batch_size= batch_size//2, shuffle= True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print ("-------------------------------------------------------------")
print ("✅Completed tranform to tensor✅")
print (f"Shape of train: x={x_men_train_t.shape}; y ={y_men_train_t.shape}; t={t_men_train_t.shape}")
print (f"Shape of val: x={x_men_val_t.shape}; y={y_men_val_t.shape}; t={t_men_val_t.shape}")
print (f"Shape of test: x={x_men_test_t.shape}; y={y_men_test_t.shape}; t={t_men_test_t.shape}")



Evaluation metrics

In [ ]:
from metrics import auuc, auqc, lift, krcc

Build Model

In [ ]:
from dragonnet import Dragonnet

In [ ]:
print("📊 Data Distribution Check:")
print(f"Y train: mean={y_men_train.mean():.4f}, std={y_men_train.std():.4f}")
print(f"Y train zeros: {(y_men_train == 0).sum()} / {len(y_men_train)} ({(y_men_train == 0).sum()/len(y_men_train)*100:.1f}%)")
print(f"\nTreatment balance:")
print(f"  Train: {(t_men_train == 1).sum()} treated, {(t_men_train == 0).sum()} control")
print(f"  Test:  {(t_men_test == 1).sum()} treated, {(t_men_test == 0).sum()} control")

In [ ]:
# seed = seed_everything_random()
# print(f"Using seed: {seed}")

# print(f"Experiment completed with seed: {seed}")

In [ ]:
seed = 10
seed_everything(seed)

In [ ]:
# Cài đặt Optuna (chỉ cần chạy 1 lần)
!pip install optuna

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def objective(trial):
    """
    Objective function cho Optuna để tối ưu theo validation loss
    Tối ưu: learning_rate, weight_decay, alpha, beta
    """
    # Suggest hyperparameters
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
    
    # Thêm alpha và beta vào tối ưu với range hợp lý
    alpha = trial.suggest_float('alpha', 0.0, 2.0)
    beta = trial.suggest_float('beta', 0.0, 2.0)
    
    # Set seed cho reproducibility
    seed_everything(seed)
    
    # Khởi tạo model với hyperparameters được suggest
    model = Dragonnet(
        input_dim=x_men_train_t.shape[1],
        epochs=30,  # Giảm epochs để tối ưu nhanh hơn
        alpha=alpha,
        beta=beta,
        learning_rate=lr,
        weight_decay=weight_decay
    )
    
    # Train model
    model.fit(train_t_loader, train_c_loader, val_loader)
    
    # Evaluate trên validation set - TRẢ VỀ LOSS
    val_loss = model.validate(val_loader)
    
    # Report intermediate value để có thể dừng sớm các trial không tốt
    trial.report(val_loss, step=30)
    
    # Handle pruning: dừng trial nếu không promising
    if trial.should_prune():
        raise optuna.TrialPruned()
    
    return val_loss

print("✅ Objective function đã được định nghĩa!")

In [ ]:
# Tạo Optuna study - MINIMIZE validation loss
study = optuna.create_study(
    direction='minimize',  # MINIMIZE validation loss
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10),
    study_name='dragonnet_full_optimization'
)

# Chạy optimization
print("🚀 Bắt đầu tối ưu hyperparameters (LR, WD, Alpha, Beta)...")
print("=" * 70)
study.optimize(objective, n_trials=30, show_progress_bar=True)

# In kết quả tốt nhất
print("\n" + "=" * 70)
print("✅ KẾT QUẢ TỐI ƯU:")
print("=" * 70)
print(f"🏆 Best Validation Loss: {study.best_value:.4f}")
print(f"\n📊 Best Hyperparameters:")
for param, value in study.best_params.items():
    print(f"  • {param}: {value:.6f}")
print("=" * 70)

In [ ]:
# Visualize optimization history
fig1 = plot_optimization_history(study)
fig1.update_layout(
    title="Optimization History (Validation Loss)",
    xaxis_title="Trial",
    yaxis_title="Validation Loss (Lower is Better)",
    height=500
)
fig1.show()

# Visualize parameter importances
fig2 = plot_param_importances(study)
fig2.update_layout(
    title="Hyperparameter Importance",
    height=500
)
fig2.show()

In [ ]:
# Train final model với best hyperparameters
best_lr = study.best_params['learning_rate']
best_wd = study.best_params['weight_decay']
best_alpha = study.best_params['alpha']
best_beta = study.best_params['beta']

seed_everything(seed)

dragonnet_optimized = Dragonnet(
    input_dim=x_men_train_t.shape[1],
    epochs=100,  # Full epochs
    alpha=best_alpha,
    beta=best_beta,
    learning_rate=best_lr,
    weight_decay=best_wd
)

print(f"🚀 Training với Best Hyperparameters:")
print("=" * 70)
print(f"  • Learning Rate:  {best_lr:.2e}")
print(f"  • Weight Decay:   {best_wd:.2e}")
print(f"  • Alpha:          {best_alpha:.4f}")
print(f"  • Beta:           {best_beta:.4f}")
print("=" * 70)

dragonnet_optimized.fit(train_t_loader, train_c_loader, val_loader)
print("\n✅ Complete training với hyperparameters tối ưu!")

In [ ]:
# Evaluate model với hyperparameters tối ưu
print("📊 Evaluating optimized model...")
y0_pred, y1_pred, _, _ = dragonnet_optimized.predict(x_men_test_t)

uplift_pred = (y1_pred - y0_pred).numpy().flatten()
y_true = y_men_test_t.numpy().flatten()
t_true = t_men_test_t.numpy().flatten()

auuc_opt = auuc(y_true, t_true, uplift_pred, bins=100, plot=True)
auqc_opt = auqc(y_true, t_true, uplift_pred, bins=100, plot=True)
lift_opt = lift(y_true, t_true, uplift_pred, h=0.3)
krcc_opt = krcc(y_true, t_true, uplift_pred, bins=100)

print("\n" + "=" * 70)
print("📈 METRICS với Optimized Hyperparameters:")
print("=" * 70)
print(f"  AUUC: {auuc_opt:.4f}")
print(f"  AUQC: {auqc_opt:.4f}")
print(f"  Lift: {lift_opt:.4f}")
print(f"  KRCC: {krcc_opt:.4f}")
print(f"  Seed: {seed}")
print("=" * 70)